# 🚀 Fabric Solution Accelerator Deployment Notebook

This notebook orchestrates the end-to-end deployment of **Advanced Metering Infrastructure (AMI)** solution assets into the current Microsoft Fabric workspace using the `fabric-cicd` library.

## This notebook performs the following tasks:
1. **📦 Package Installation**: Install required libraries and dependencies
1. **⚙️ Parameter Configuration and Library Import:** Configure parameters and import required libraries
1. **📥 Source Code Download**: Download and extracts solution content from GitHub repository
1. **🚀 Fabric Item Deployment**: Deploy Fabric items and map them to each other to preserve dependencies
1. **✅ Post-Deployment Tasks**: Complete post-deployment configuration tasks.


## 📦 Package Installation

In [ ]:
%pip install fabric-cicd --quiet
%pip install --upgrade azure-core azure-identity --quiet

In [ ]:
print("⚠️ Restarting Python kernel for installed packages to take effect")
notebookutils.session.restartPython()

## ⚙️ Parameter Configuration and Library Import

Update these values to customize the deployment for your environment:

In [ ]:
# Define user-configurable parameters
DEFAULT_API_ROOT_URL = "https://api.fabric.microsoft.com" #Default is https://api.fabric.microsoft.com, but may vary depending on your environment
DEBUG = False

In [ ]:
# The following settings should not be modified by the user
# GitHub Repository Configuration
repo_owner = "slavatrofimov"
repo_name = "Real-Time-Grid-Intelligence-with-Microsoft-Fabric"
branch = "main"
folder_to_extract = "workspace"

# GitHub Personal Access Token (if required for private repositories)
github_token = ""

# Deployment Configuration
deployment_environment = "DEV"  # Options: DEV, TEST, PROD

# File System Paths
path_prefix = '.lakehouse/default/Files'

extract_to_directory = path_prefix + "/src"
repository_directory = extract_to_directory + "/" + folder_to_extract

In [ ]:
### Import libraries
import subprocess
import os
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import sempy.fabric as fabric
import base64
from pathlib import Path
from typing import Optional, Any
from datetime import datetime, timezone
from azure.core.credentials import TokenCredential, AccessToken
import fabric_cicd.constants
from fabric_cicd import FabricWorkspace, publish_all_items

## 📥 Source Code Download

Download and extract solution source files and configuration from the GitHub repository:

In [ ]:
# Download and extract GitRepository to a folder

def download_and_extract_folder(repo_owner, repo_name, github_token, extract_to, branch="main", 
                               folder_to_extract="workspace", remove_folder_prefix=""):
    """
    Download a GitHub repository and extract a specific folder directly to disk
    without saving the zip file.
    
    Args:
        repo_owner: GitHub repository owner
        repo_name: GitHub repository name
        github_token: GitHub personal access token (if required)
        extract_to: Local directory to extract files to
        branch: Git branch to download (default: "main")
        folder_to_extract: Folder path within the repo to extract
        remove_folder_prefix: Prefix to remove from extracted file paths
    """
    try:
        # Construct the URL for the GitHub API to download the repository as a zip file
        url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"

        # Set up headers for authentication if a token is provided
        headers = {}
        if github_token:
            headers['Authorization'] = f"token {github_token}"
            headers["Accept"] = "application/vnd.github.v3+json"

        # Make a request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Delete target directory if exists
        if os.path.exists(extract_to) and os.path.isdir(extract_to):
            shutil.rmtree(extract_to)
            print(f'Deleted existing directory: {extract_to}')
        
        # Ensure the extraction directory exists
        os.makedirs(extract_to, exist_ok=True)
        
        # Process the zip file directly from memory
        with zipfile.ZipFile(BytesIO(response.content)) as zipf:
            for file_info in zipf.infolist():
                # Check if the file is in the folder we want to extract
                normalized_path = re.sub(r'^.*?/', '/', file_info.filename)
                
                if normalized_path.startswith(f"/{folder_to_extract}"):
                    # Calculate the output path
                    parts = file_info.filename.split('/')
                    relative_path = '/'.join(parts[1:])  # Remove repo root folder
                    
                    # Remove the specified prefix if provided
                    if remove_folder_prefix:
                        relative_path = relative_path.replace(remove_folder_prefix, "", 1)
                    
                    output_path = os.path.join(extract_to, relative_path)
                    
                    # Skip if it's a directory entry
                    if file_info.filename.endswith('/'):
                        os.makedirs(output_path, exist_ok=True)
                        continue
                    
                    # Ensure the directory for the file exists
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    
                    # Extract and write the file
                    with zipf.open(file_info) as source_file:
                        with open(output_path, 'wb') as target_file:
                            target_file.write(source_file.read())
                            
        print(f"Successfully extracted {folder_to_extract} from {repo_owner}/{repo_name} to {extract_to}")
        
    except Exception as e:
        error_msg = f"A {type(e).__name__} error occurred. This error may be intermittent. Consider stopping the current notebook session and re-running the notebook again."
        print(error_msg)
        #  Re-raise the exception 
        raise

# Execute repo download and extraction using configured parameters
print(f"📥 Downloading {repo_name} from {repo_owner}/{repo_name}:{branch}")
print(f"📁 Extracting '{folder_to_extract}' folder to '{extract_to_directory}'")

download_and_extract_folder(
    repo_owner=repo_owner,
    repo_name=repo_name,
    github_token=github_token,
    extract_to=extract_to_directory,
    branch=branch,
    folder_to_extract=folder_to_extract,
    remove_folder_prefix=""
)

print("✅ Source code download and extraction completed successfully")

## 🚀 Fabric Item Deployment

Deploy solution assets to the current Fabric workspace:

In [ ]:
# Custom Token Credential class for authentication in a Fabric Notebook
# This class enables the fabric-cicd library to be run in a Fabric notebook

class FabricNotebookTokenCredential(TokenCredential):
    """Token credential for Fabric Notebooks using notebookutils authentication."""
    
    def get_token(self, *scopes: str, claims: Optional[str] = None, tenant_id: Optional[str] = None,
                  enable_cae: bool = False, **kwargs: Any) -> AccessToken:
        """Get access token from Fabric notebook environment."""
        access_token = notebookutils.credentials.getToken("pbi")       
        expiration = self._extract_jwt_expiration(access_token)
        return AccessToken(token=access_token, expires_on=expiration)
    
    def _extract_jwt_expiration(self, token: str) -> int:
        """Extract expiration timestamp from JWT token."""
        try:
            # Split JWT and get payload (middle part)
            payload_b64 = token.split(".")[1]           
            # Add padding if needed for base64 decoding
            payload_b64 += "=" * (-len(payload_b64) % 4)
            # Decode and parse payload
            payload_bytes = base64.urlsafe_b64decode(payload_b64.encode("utf-8"))
            payload = json.loads(payload_bytes.decode("utf-8"))
            # Extract expiration claim
            exp = payload.get("exp")
            if exp is None:
                raise ValueError("JWT missing expiration claim")
            return exp
        except (IndexError, json.JSONDecodeError, ValueError) as e:
            raise ValueError(f"Invalid JWT token format: {e}")

In [ ]:
# Configure constants for fabric-cicd library
fabric_cicd.constants.DEFAULT_API_ROOT_URL = DEFAULT_API_ROOT_URL

# Get current workspace information
client = fabric.FabricRestClient()
workspace_id = fabric.get_workspace_id()
print(f"Target workspace ID: {workspace_id}")

# Enable debugging for more verbose execution logging
if DEBUG:
    from fabric_cicd import change_log_level
    change_log_level("DEBUG")

# Function to execute deployment of all specified item types
def deploy_artifacts(target_workspace):
    print("🚀 Starting deployment of Fabric items...")
    print(f"📋 Item types in scope: {', '.join(target_workspace.item_type_in_scope)}")
    publish_all_items(target_workspace)
    print("✅ Deployment completed successfully!")

# Initialize the FabricWorkspace object with configured parameters
target_workspace = FabricWorkspace(
    workspace_id=workspace_id,
    environment=deployment_environment,
    repository_directory=repository_directory,
    token_credential=FabricNotebookTokenCredential()
)

In [ ]:
# Start by deploying data stores
item_types_in_scope = [
    "Eventhouse", 
    "KQLDatabase", 
    "Lakehouse"
]

target_workspace.item_type_in_scope=item_types_in_scope

# Execute deployment of all specified item types
deploy_artifacts(target_workspace)

In [ ]:
# Deploy remaining items
item_types_in_scope = [
    "Eventstream", 
    "Notebook", 
    "KQLDashboard", 
    "SemanticModel", 
    "Report", 
    "Reflex", 
    "DataAgent"
]

target_workspace.item_type_in_scope=item_types_in_scope

# Execute deployment of all specified item types
deploy_artifacts(target_workspace)

### 🗺️ Deploy Service Area Map
Deploy the Service Area Map with artifact ID remapping:

In [ ]:
# Deploy Service Area Map with artifact ID remapping
import time

# Define a function to get the id of the destination folder
def get_folder_id_by_name(folder_name: str, workspace_id: str) -> str | None:
    """Get folder ID by display name, returns None if not found."""
    try:

        url = f'v1/workspaces/{workspace_id}/folders'
        folders = client.get(url)
        for folder in folders.json()["value"]:
            if folder["displayName"] == folder_name:
                return(folder["id"])
        
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_item_definition_from_repo(item_relative_path):
    """Load item definition files from the extracted repository."""
    item_path = os.path.join(repository_directory, item_relative_path)
    
    # Read .platform file for metadata
    platform_file = os.path.join(item_path, '.platform')
    with open(platform_file, 'r', encoding='utf-8') as f:
        platform_data = json.load(f)
    
    return platform_data, item_path


def remap_map_artifact_ids(map_definition, workspace_id):
    """
    Remap artifact IDs in the map definition to match the current workspace.
    Updates lakehouses, kqlDatabases, and parentArtifactIds in layer sources.
    """
    print("🔧 Remapping artifact IDs in map definition...")
    
    # Build artifact ID mapping
    artifact_mapping = {}
    
    # Remap lakehouses
    if 'lakehouses' in map_definition.get('dataSources', {}):
        for lakehouse in map_definition['dataSources']['lakehouses']:
            old_artifact_id = lakehouse['artifactId']
            
            # Get the actual lakehouse ID in this workspace
            try:
                new_artifact_id = fabric.resolve_item_id('ReferenceDataLH', 'Lakehouse')
                artifact_mapping[old_artifact_id] = new_artifact_id
                
                # Update the map definition
                lakehouse['workspaceId'] = workspace_id
                lakehouse['artifactId'] = new_artifact_id
                print(f"  ✓ Remapped Lakehouse: {old_artifact_id} → {new_artifact_id}")
            except Exception as e:
                print(f"  ⚠️ Warning: Could not resolve Lakehouse ID: {e}")
    
    # Remap KQL Databases
    if 'kqlDataBases' in map_definition.get('dataSources', {}):
        for kql_db in map_definition['dataSources']['kqlDataBases']:
            old_artifact_id = kql_db['artifactId']
            
            # Get the actual KQL Database ID in this workspace
            try:
                new_artifact_id = fabric.resolve_item_id('PowerUtilitiesEH', 'KQLDatabase')
                artifact_mapping[old_artifact_id] = new_artifact_id
                
                # Update the map definition
                kql_db['workspaceId'] = workspace_id
                kql_db['artifactId'] = new_artifact_id
                print(f"  ✓ Remapped KQL Database: {old_artifact_id} → {new_artifact_id}")
            except Exception as e:
                print(f"  ⚠️ Warning: Could not resolve KQL Database ID: {e}")
    
    # Remap parentArtifactIds in layer sources
    if 'layerSources' in map_definition:
        for layer_source in map_definition['layerSources']:
            if 'parentArtifactId' in layer_source and layer_source['parentArtifactId']:
                old_parent_id = layer_source['parentArtifactId']
                
                # Look up the new ID from our mapping
                if old_parent_id in artifact_mapping:
                    new_parent_id = artifact_mapping[old_parent_id]
                    layer_source['parentArtifactId'] = new_parent_id
                    print(f"  ✓ Remapped Layer Source '{layer_source['name']}': {old_parent_id} → {new_parent_id}")
                else:
                    print(f"  ⚠️ Warning: No mapping found for parentArtifactId in layer '{layer_source['name']}': {old_parent_id}")
    
    print(f"✅ Artifact ID remapping completed ({len(artifact_mapping)} mappings)")
    return map_definition


def deploy_map_item(map_name, map_relative_path):
    """
    Deploy a Map item using the Fabric REST API with artifact ID remapping.
    Reference: https://learn.microsoft.com/en-us/rest/api/fabric/map/items/create-map
    """
    print(f"\n{'='*60}")
    print(f"📍 Deploying Map: {map_name}")
    print(f"{'='*60}")
    
    try:
        # Get map definition from repository
        platform_data, map_path = get_item_definition_from_repo(map_relative_path)
        
        display_name = platform_data['metadata']['displayName']
        description = platform_data['metadata'].get('description', '')
        
        # Read map.json file
        map_json_file = os.path.join(map_path, 'map.json')
        with open(map_json_file, 'r', encoding='utf-8') as f:
            map_definition = json.load(f)
        
        # Remap artifact IDs to match current workspace
        map_definition = remap_map_artifact_ids(map_definition, workspace_id)
        
        # Step 1: Create the Map item
        print(f"🔧 Creating Map item: {display_name}")
        create_url = f'v1/workspaces/{workspace_id}/maps'
        
        create_payload = {
            "displayName": display_name,
            "description": description
        }
        
        create_response = client.post(create_url, json=create_payload)
        
        if create_response.status_code in [200, 201]:
            map_item = create_response.json()
            map_id = map_item['id']
            print(f"✅ Map item created successfully (ID: {map_id})")
        elif create_response.status_code == 409:
            print(f"ℹ️ Map '{display_name}' already exists, retrieving existing item...")
            map_id = fabric.resolve_item_id(display_name, 'Map')
            print(f"✅ Using existing Map (ID: {map_id})")
        else:
            raise Exception(f"Failed to create Map: {create_response.status_code} - {create_response.text}")
        
        # Step 2: Update Map definition with remapped artifact IDs
        print(f"🔧 Updating Map definition with remapped artifact IDs...")
        update_url = f'v1/workspaces/{workspace_id}/maps/{map_id}/updateDefinition'
        
        # Prepare definition payload according to API spec
        definition_payload = {
            "definition": {
                "parts": [
                    {
                        "path": "map.json",
                        "payload": base64.b64encode(json.dumps(map_definition).encode('utf-8')).decode('utf-8'),
                        "payloadType": "InlineBase64"
                    }
                ]
            }
        }
        
        update_response = client.post(update_url, json=definition_payload)
        
        if update_response.status_code in [200, 202]:
            print(f"✅ Map definition updated successfully")
            
            # If async operation, poll for completion
            if update_response.status_code == 202:
                print(f"⏳ Waiting for definition update to complete...")
                time.sleep(5)  # Give it time to process
                
        else:
            print(f"⚠️ Warning: Map definition update returned status {update_response.status_code}")
            print(f"Response: {update_response.text[:500]}")
        
        # Step 3: Move to correct folder
        target_folder_id = get_folder_id_by_name('Visualize and Chat', workspace_id)
        if target_folder_id:
            print(f"🔧 Moving Map to 'Visualize and Chat' folder...")
            move_url = f'v1/workspaces/{workspace_id}/items/{map_id}/move'
            move_payload = {"targetFolderId": target_folder_id}
            move_response = client.post(move_url, json=move_payload)
            
            if move_response.status_code == 200:
                print(f"✅ Map moved to folder successfully")
            else:
                print(f"⚠️ Could not move Map to folder: {move_response.status_code}")
        
        print(f"✅ Map deployment completed: {display_name}")
        return map_id
        
    except Exception as e:
        print(f"❌ Error deploying Map: {e}")
        raise


# Deploy the Service Area Map
try:
    map_id = deploy_map_item(
        map_name="Service Area Map",
        map_relative_path="Visualize and Chat/Service Area Map.Map"
    )    
    print("✅ Service Area Map deployed successfully!")
    
except Exception as e:
    print(f"\n❌ Map deployment failed: {e}")
    print("ℹ️ Check the logs above for details.")

## ✅ Post-Deployment Tasks

### 🔧 Automated Post-Deployment Tasks

In [ ]:
# Move the KQL Database to the same subfolder as the Eventhouse
# To compensate for the unexpected placement of the item in the initial deployment

# Get current workspace information
client = fabric.FabricRestClient()
workspace_id = fabric.get_workspace_id()
eventhouse_id = fabric.resolve_item_id('PowerUtilitiesEH', 'Eventhouse')

# Step 1: move eventhose to the root folder
url = f'v1/workspaces/{workspace_id}/items/{eventhouse_id}/move'

# Define payload for the API call and execute the call
payload = {} # Move to root folder
client.post(url, json = payload)

# Step 2: move eventhose (with the child KQL database) to the desired destination folder
# Get target folder
target_folder_id = get_folder_id_by_name('Store and Query', workspace_id)

# Define payload for the API call and execute the call
payload = {
  "targetFolderId": f"{target_folder_id}"
}

client.post(url, json = payload)
print("✅ Eventhouse moved successfully!")

In [ ]:
# Download and save sample data files from the GitHub repository to the lakehouse

# Get abfs path to the RfeferenceDataLH lakehouse and mount it as a local filesystem
abfsPath = notebookutils.lakehouse.getWithProperties('ReferenceDataLH').properties['abfsPath']
notebookutils.fs.mount(abfsPath,"/ReferenceDataLH") # Mount 

# Define the path to the local directory where the file will be saved
target_directory = notebookutils.fs.getMountPath("/ReferenceDataLH") + "/Files/data/"

# Function to download a file from GitHub repository and save it locally
def download_file(repo_owner, repo_name, github_token, branch, file_path, target_directory):
    """
    Download and save a file from a GitHub repository.
    
    Args:
        repo_owner: GitHub repository owner
        repo_name: GitHub repository name
        github_token: GitHub personal access token (if required)
        branch: Git branch to download (default: "main")
        file_path: File path within the repo to download
        target_directory: Directory where to save the file
    """

    # Download sample data file
    file_url = f"https://raw.githubusercontent.com/{repo_owner}/{repo_name}/refs/heads/{branch}/{file_path}"
    file_name = file_url.split("/")[-1]  # Extract filename from URL

    try:
        # Create target directory if it doesn't exist
        os.makedirs(target_directory, exist_ok=True)
        
        # Set up headers for authentication if a token is provided
        headers = {}
        if github_token:
            headers['Authorization'] = f"token {github_token}"
            headers["Accept"] = "application/vnd.github.v3+json"

        # Download the file
        print(f"📥 Downloading file from {file_url}")
        response = requests.get(file_url, headers=headers)
        response.raise_for_status()
        
        # Save to target directory
        target_path = os.path.join(target_directory, file_name)
        with open(target_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ File saved successfully to {target_path}")
        
    except requests.RequestException as e:
        print(f"❌ Error downloading file: {e}")
        raise
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        raise

# Define the paths to the desired files in the repo
file_paths = ["data/vehicle_route_points.json", "data/transmission_lines.geojson"]

for file_path in file_paths:
    download_file(repo_owner, repo_name, github_token, branch, file_path, target_directory)

### ⚙️ Manual Post-Deployment Steps
Complete the following tasks to finish the installation
1. Refresh your browser window to reflect the newly-deployed Fabric items
1. Generate reference data by opening and running the **AMI Reference Data Simulation** notebook (in the Simulation folder)